In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from textblob import TextBlob
import torch
from torch import nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/Users/abhishekvaidyanathan/Desktop/RedPILS/redpils/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


device(type='cpu')

In [6]:
# start here
train_df = pd.read_csv('train_full.csv')
eval_df = pd.read_csv('eval_full.csv')

In [53]:
train_df['true_label'] = train_df.apply(lambda x: np.sign(x['target'])+1, axis=1)
train_df.head()

,clean_title,clean_comment,comment_id,url,target,true_label
0,BJP Gujarat 2022 Manifesto.,There are two types of parties those that prom...,iy20u35,/r/IndiaSpeaks/comments/z5u6kf/bjp_gujarat_202...,0.00,1.0
1,Rajasthan BJP MLA Sanjay Sharma posted this on...,"If fundamentals are sound then, faith can help...",hnc96s6,/r/IndiaSpeaks/comments/r9df3s/rajasthan_bjp_m...,0.75,2.0
2,The new Indian If Trump's supporters blame Ant...,"Contrary to popular belief, until Modi, India ...",giqnuc2,/r/india/comments/ku5i9f/the_new_indian_if_tru...,-0.50,0.0
3,Kejriwal says that Mukesh Ambani's Antilia is ...,Bro inhone bahut land hijack kiya hai kisse pa...,iq3l8o5,/r/IndiaSpeaks/comments/xpb83d/kejriwal_says_t...,-0.25,0.0
4,The picture pretty much sums up the ideology o...,The 1.9 million denied citizenship in Assam al...,fbtqycg,/r/india/comments/ee11sp/the_picture_pretty_mu...,-1.00,0.0


In [55]:
eval_df['true_label'] = eval_df.apply(lambda x: np.sign(x['target'])+1, axis=1)
eval_df.head()

,clean_title,clean_comment,comment_id,url,target,true_label
0,ResignModi trending in India on 1 with 200k tw...,"People tend to look down on ""social media acti...",gwe056n,/r/india/comments/n11sqc/resignmodi_trending_i...,-0.5,0.0
1,"Put 'The Kashmir Files' on YouTube, everyone w...","I remember it was made to release twice, and t...",i1y3pv3,/r/india/comments/tmcun4/put_the_kashmir_files...,0.0,1.0
2,Kejriwal says 'The Kashmir Files' is a jhoothi...,Bhai delhi m hindu bht h but ye aur caste k lo...,i21haf8,/r/IndiaSpeaks/comments/tmhqqg/kejriwal_says_t...,0.5,2.0
3,Government of India has Blacklisted Karl Rock ...,mahatma gandhi's shadows are fading. retards w...,h4lazw7,/r/india/comments/ogrc0d/government_of_india_h...,-1.0,0.0
4,Kejriwal says 'The Kashmir Files' is a jhoothi...,Isn't this kejru the same guy who kept tweetin...,i1yrkwx,/r/IndiaSpeaks/comments/tmhqqg/kejriwal_says_t...,0.5,2.0


In [56]:
right = """right wing, RW, authority, hierarchy, order, duty, tradition, reaction, nationalism, conservative, right-libertarian, \
neoconservative, imperialist, monarchist, fascist, reactionaries, traditionalist, traditional, death penalty, \
religion, Bhajpa, BJP, Shiv Sena, RSS, MNS, Sanatan, dharm, Hindutva, Islamophobia, Narendra, Modi, Amit, Shah, \
mandir, ram, valmiki, ramayan, Bharatiya, Janata, Democratic Alliance, NDA, AIADMK, Janta Dal, bhakt, CAA, NRC, hindu majority, \
hindu unity, hindu pride, nationalist, sangh, sanghi, yogi, brahmin, brahman, smriti irani, hindu rashtra, jai shri ram, \
pm cares, pmcares, adani, hindu""".lower()
left = """left wing, LW, leftists, freedom, equality, fraternity, rights, progress, reform, internationalism, anarchist, communist, socialist, \
democratic socialist, social democrat, left-libertarian, progressive, social, liberal, western, Congress, UPA, RG, mamata, \
Aam, aadmi, AAP, CPI, Welfare, Protectionism, Commies, Rahul, gandhi, indira, yatra, arvind, kejriwal, inclusivity, \
libby, libbies, sjw, libtard, hinduphobia, LGBTQ, masjid, pappu, christian, muslim, secular, minority, minorities, Shashi, Tharoor, \
gay, lesbian, transgender, trans, reservation, abrahamic, godi""".lower()

right_terms = set(right.split(', '))
left_terms = set(left.split(', '))
len(right_terms), len(left_terms)

(62, 60)

In [57]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer

In [58]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [59]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [70]:
x_train = np.array(train_df['clean_comment'])
y_train = np.array(train_df['true_label'])
x_val = np.array(eval_df['clean_comment'])
y_val = np.array(eval_df['true_label'])

In [71]:
# Create a tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on your text
# tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)

In [72]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 50  # Only consider the first 200 words of each movie review
# (x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

1999 Training sequences
1000 Validation sequences


In [73]:
embed_dim = 32  # Embedding size for each token
num_heads = 8  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(3, activation="softmax")(x)
outputs = (outputs - 1) * 2

model = keras.Model(inputs=inputs, outputs=outputs)

In [74]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
)

Epoch 1/2
63/63 [==============================] - 5s 38ms/step - loss: 1.0986 - accuracy: 0.2881 - val_loss: 1.0986 - val_accuracy: 0.3350
Epoch 2/2
63/63 [==============================] - 2s 34ms/step - loss: 1.0986 - accuracy: 0.2996 - val_loss: 1.0986 - val_accuracy: 0.3350
